In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
import typing as t
from fdua_competition.vectorstore import build_vectorstore
from fdua_competition.enums import Mode, VectorStoreOption
from fdua_competition.rag import ResearchAssistant, RAG
from fdua_competition.enums import ChatModelOption
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.vectorstores.base import VectorStore
from pydantic import BaseModel, Field

from langchain_community.document_loaders import PyPDFium2Loader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from pprint import pprint


In [51]:
# for doc in PyPDFium2Loader(path).lazy_load():
path = '/fdua-competition/.fdua-competition/validation/documents/6.pdf'
doc = PyPDFium2Loader(path).lazy_load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=20000,  # チャンクのサイズ (文字数)
    chunk_overlap=0,  # チャンクの重複サイズ (文字数)
)

print(type(doc))
item_list = [item for item in doc]
print(type(item_list[0].page_content))

documents = [Document(page_content=item.page_content) for item in doc]
print(type(documents[0]))
chunks = text_splitter.split_documents(documents)

# chunks = text_splitter.split_documents(doc)

<class 'langchain_core.documents.base.Document'>


In [44]:
print(type(doc))

<class 'generator'>


In [49]:
print(type(item_list[0].page_content))

<class 'str'>


In [47]:
pprint(item_list[0].page_content)

('今年、当社グループとして初めて統合報告書を発行いたしました。\r\n'
 'この統合報告書をお読みいただく際のガイドマップとしてご参照いただければ幸いです。\r\n'
 '読者の皆さまへ\r\n'
 '編集方針\r\n'
 '本報告書のガイドマップ\r\n'
 '編集にあたっての考慮事項\r\n'
 '発行の目的\r\n'
 '本報告書は、株主・投資家の皆さまをはじめとする、さまざまなステークホルダーの皆さまとの対話を\r\n'
 '促進し、広く社会からの信頼を得ることを目的に制作しています。長期経営ビジョン「LIVZON DREAM \r\n'
 '2030 1st half!」を実現するための中期戦略とESG経営への取り組みのご紹介を通じて、当社グルー\r\n'
 'プの現在と中長期的な将来に関心をお持ちの皆さまにとって有用な内容とするよう努めています。\r\n'
 '初めて発行する統合報告書であるため、\r\n'
 '当社グループのビジネスモデルや特徴\r\n'
 'を丁寧に説明し、ステークホルダーの皆\r\n'
 'さまに当社グループに対するご理解と\r\n'
 'ご関心を深めていただくことを目指しま\r\n'
 'した。\r\n'
 '1 当社グループを知っていただく 2 価値観・ビジョンを具体化する 3 情報開示の質向上を目指す\r\n'
 '創業当時からの快適な環境づくりへの\r\n'
 'コミットメント、経営理念やビジョンと、\r\n'
 '経営戦略との関係性をご説明すること\r\n'
 'で、当社グループがいかに事業を通じて\r\n'
 '価値観・ビジョンを実現しているか明ら\r\n'
 'かにすることを心掛けました。\r\n'
 '発行初年度の報告書として、当社グルー\r\n'
 'プの独自性と他社との比較可能性のバ\r\n'
 'ランスに配慮した開示を心がけました。\r\n'
 'ステークホルダーの皆さまのご感想や\r\n'
 'ご意見を受けて、今後も開示の質の向\r\n'
 '上に取り組んでいきたいと考えています。\r\n'
 '① 会社の方向性を示すトップメッセージ\r\n'
 '会社を支える ガバナンス\r\n'
 '社是\r\n'
 '経営理念\r\n'
 'LIVZON\r\n'
 'ブランドステートメント\r\n'
 '「総合た

In [45]:
item_list = [item for item in doc]


[Document(metadata={'source': '/fdua-competition/.fdua-competition/validation/documents/6.pdf', 'page': 1}, page_content='今年、当社グループとして初めて統合報告書を発行いたしました。\r\nこの統合報告書をお読みいただく際のガイドマップとしてご参照いただければ幸いです。\r\n読者の皆さまへ\r\n編集方針\r\n本報告書のガイドマップ\r\n編集にあたっての考慮事項\r\n発行の目的\r\n本報告書は、株主・投資家の皆さまをはじめとする、さまざまなステークホルダーの皆さまとの対話を\r\n促進し、広く社会からの信頼を得ることを目的に制作しています。長期経営ビジョン「LIVZON DREAM \r\n2030 1st half!」を実現するための中期戦略とESG経営への取り組みのご紹介を通じて、当社グルー\r\nプの現在と中長期的な将来に関心をお持ちの皆さまにとって有用な内容とするよう努めています。\r\n初めて発行する統合報告書であるため、\r\n当社グループのビジネスモデルや特徴\r\nを丁寧に説明し、ステークホルダーの皆\r\nさまに当社グループに対するご理解と\r\nご関心を深めていただくことを目指しま\r\nした。\r\n1 当社グループを知っていただく 2 価値観・ビジョンを具体化する 3 情報開示の質向上を目指す\r\n創業当時からの快適な環境づくりへの\r\nコミットメント、経営理念やビジョンと、\r\n経営戦略との関係性をご説明すること\r\nで、当社グループがいかに事業を通じて\r\n価値観・ビジョンを実現しているか明ら\r\nかにすることを心掛けました。\r\n発行初年度の報告書として、当社グルー\r\nプの独自性と他社との比較可能性のバ\r\nランスに配慮した開示を心がけました。\r\nステークホルダーの皆さまのご感想や\r\nご意見を受けて、今後も開示の質の向\r\n上に取り組んでいきたいと考えています。\r\n① 会社の方向性を示すトップメッセージ\r\n会社を支える ガバナンス\r\n社是\r\n経営理念\r\nLIVZON\r\nブランドステートメント\r\n「総合たてものサービス企業」\r\n財務・非財務ハイライ

In [22]:
for doc in PyPDFium2Loader(path).lazy_load():
    print(type(doc))
    chunks = text_splitter.split_documents(doc)
    print(type(chunks))
    break

<class 'langchain_core.documents.base.Document'>


AttributeError: 'tuple' object has no attribute 'page_content'

In [3]:
# RAGクラスを継承して, 新しいRAGを作成する例

## レスポンスの型式を定義
class FactCheckerResponse(BaseModel):
    score: str = Field(description="スコア")
    reason: str = Field(description="理由")
    reference: list[str] = Field(description="[参照した文章]")


# RAGクラスを継承して新しいRAGクラスを定義
# prompt_template(), build_payload(), output_structure() メソッドをオーバーライドする
class FactChecker(RAG):
    def __init__(
        self, vectorstore: VectorStore, chat_model_option: ChatModelOption = ChatModelOption.AZURE, language: str = "japanese"
    ):
        super().__init__(vectorstore=vectorstore, chat_model_option=chat_model_option)
        self.language = language

    @property
    def prompt_template(self) -> ChatPromptTemplate:
        """プロンプトのテンプレート"""
        return ChatPromptTemplate.from_messages(
            [
                ("system", "情報の正確さを与えられたコンテキストのみ使ってを10点満点で評価して. {language}"),
                ("system", "コンテキスト:\n{context}"),
                ("user", "チェック対象:\n{llm_response}"),
            ]
        )

    def build_payload(self, query: str) -> dict[str, t.Any]:
        """プロンプトテンプレートをこの辞書で埋める"""
        return {"llm_response": query, "language": self.language}

    @property
    def output_structure(self) -> BaseModel:
        """出力型式を定義する"""
        return FactCheckerResponse


# [end: research_assistant]

In [4]:
vectorstore = build_vectorstore("one_assistant_test", Mode.TEST, VectorStoreOption.CHROMA)

[prepare_vectorstore] chroma: /fdua-competition/.fdua-competition/vectorstores/chroma
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/9.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/8.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/10.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/6.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/7.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/5.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/4.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-co

In [5]:
assistant = ResearchAssistant(vectorstore)

In [6]:
res = assistant.invoke("電通グループの創業は何年ですか？")
print(res)

query='電通グループの創業は何年ですか？' response='電通グループは1901年に創業されました' reason='創業年は文書内で明記されており1901年と記されています' organization_name='電通グループ' contexts=['株式会社日本広告社創業 光永星郎1901年']


In [7]:
fact_checker = FactChecker(vectorstore)

print("prompt_template:")
print(fact_checker.prompt_template)

query = f"{res.query}: {res.response}"
print(f"\nquery: {query}")

fact_checked_res = fact_checker.invoke(query)
print(f"\nres: {fact_checked_res}")

prompt_template:
input_variables=['context', 'language', 'llm_response'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='情報の正確さを与えられたコンテキストのみ使ってを10点満点で評価して. {language}'), additional_kwargs={}), SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='コンテキスト:\n{context}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['llm_response'], input_types={}, partial_variables={}, template='チェック対象:\n{llm_response}'), additional_kwargs={})]

query: 電通グループの創業は何年ですか？: 電通グループは1901年に創業されました

res: score='10' reason='提供された情報は正確で、1901年に電通グループが創業されたことが明記されています。' reference=['日本広告社創立は1901年に光永星郎によって行われた。']
